# First and Second Derivatives

## Usage

Update the constants below to work with your file structure

HYPERSPECRAL_VECTOR_CSV_PATH this should be the path to a csv file cointaining the hyperspectral data to be transformed into first and second derivative data (following all pre-precessing such as atmospheric water vapour removal). 

WAVELENGTHS_CSV_PATH this should be the path to a csv file containing the wavelengths included in the hyperspectral data. 

OUTPUT_CSV_PATH_FIRST_DERIVATIVE this will be where the generated data is placed for the first deviative outputs.

OUTPUT_CSV_PATH_SECOND_DERIVATIVE this will be where the generated data is placed for the second deviative outputs.

In [ ]:
HYPERSPECRAL_VECTOR_CSV_PATH = 'preprocessed_data/site_with_hyperspectral_data.csv'
WAVELENGTHS_CSV_PATH = 'preprocessed_data/wavelength_data.csv'
OUTPUT_CSV_PATH_FIRST_DERIVATIVE = 'transformed_data/first_derivative.csv'
OUTPUT_CSV_PATH_SECOND_DERIVATIVE = 'transformed_data/second_derivative.csv'

In [ ]:
import pandas as pd
import geopandas
data = pd.read_csv(HYPERSPECRAL_VECTOR_CSV_PATH)

data['geometry'] = geopandas.GeoSeries.from_wkt(data['geometry'])

data_gdf = geopandas.GeoDataFrame(data, geometry='geometry')

data_gdf

In [ ]:
wavelengths = pd.read_csv(WAVELENGTHS_CSV_PATH)

sample = 0

count_row = data_gdf.shape[0]

first_derivative = wavelengths
second_derivative = wavelengths

# create individual wavelength tables for each sample - use a for loop - extract the row based on the sample number
# transpose, create full table of derivatives - give the tables names based on PFT (link to PFT column) and number - if PFT == prev +1, otherwise start at 1

for num in range(count_row):
    absorption_all_cols = data_gdf.loc[[sample]]
    absorption = absorption_all_cols.iloc[:, 1:359]
    original_spectra = absorption.transpose()
    original_spectra = original_spectra.rename(columns={sample: 'Original Spectra'})
    #original_spectra = original_spectra.rename(index={0:357}) # want to rename to match the other dataframe
    original_spectra = original_spectra.reset_index()
    #derivatives = pd.DataFrame(wavelengths)
    derivatives = pd.concat([wavelengths,original_spectra], axis=1)
    derivatives.tail()
    #derivatives["First Derivative"] = ('Original Spectra' / 'Wavelength') # don't know how to write the equation here - see excel (interaction between rows as well as columns)
    derivatives["Shifted_wavelength"] = derivatives["Wavelength"].shift(-1, fill_value=0)
    derivatives["Shifted_original_spectra"] = derivatives["Original Spectra"].shift(-1, fill_value=0)
    derivatives["First_derivative"] = (derivatives["Shifted_original_spectra"] - derivatives["Original Spectra"]) / (derivatives["Shifted_wavelength"] - derivatives["Wavelength"])
    derivatives["Second_derivative"] = (derivatives["Shifted_original_spectra"] - derivatives["Original Spectra"]*2) / (derivatives["Shifted_wavelength"] - derivatives["Wavelength"])**2
    # name the table and add to two additional tables - a first derivative one and a second derivative one
    # need to improve naming of columns to include PFT and number
    first_derivative["sample"] = derivatives[["First_derivative"]]
    first_derivative = first_derivative.rename(columns = {"sample": sample})
    second_derivative["sample"] = derivatives[["Second_derivative"]]
    second_derivative = second_derivative.rename(columns = {"sample": sample})
    sample += 1


second_derivative


In [5]:
#change the name of headings to reflect PFT

first_derivative_transposed = first_derivative.transpose()
first_derivative_transposed = first_derivative_transposed.drop('Wavelength')
first_derivative_transposed

second_derivative_transposed = second_derivative.transpose()
second_derivative_transposed = second_derivative_transposed.drop('Wavelength')
second_derivative_transposed

# pft = data_gdf[['PFT']]
geometry = data_gdf[['geometry']]
# # first_derivative_transposed['PFT'] = pft
first_derivative_transposed['geometry'] = geometry

# second_derivative_transposed['PFT'] = pft
second_derivative_transposed['geometry'] = geometry

# first_derivative_transposed['geometry'] = geopandas.GeoSeries.from_wkt(first_derivative_transposed['geometry'])
# first_derivative_gdf = geopandas.GeoDataFrame(first_derivative_transposed, geometry='geometry')

# first_derivative_gdf

first_derivative_transposed.to_csv(OUTPUT_CSV_PATH_FIRST_DERIVATIVE)
second_derivative_transposed.to_csv(OUTPUT_CSV_PATH_SECOND_DERIVATIVE)